# pyEPR Calculating Dielectric Participations

Zlatko Minev

**Summary:** Following Appendix E of the energy-participation-ratio (EPR) paper, here we demonstrate how to calcualte the dielectric EPR of a chip substrate in a qubit eigen mode. 

We use the following definitions for the RMS energy stored in a volume $V$,
\begin{align}
\mathcal{E}_{\mathrm{elec}}&=&\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}\;,\\\mathcal{E}_{\mathrm{mag}}&=&\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{H}_{\text{max}}^{*}\overleftrightarrow{\mu}\vec{H}_{\text{max}}\;,
\end{align}

## The simple way  

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True

In [3]:
from pyEPR import * 

project = Project_Info(r"C:\path-to-my-dir",
                       'My file name',
                       'My design name')

project.connect()

epr_hfss = pyEPR_HFSS(project)





Connecting to HFSS ...
	File path to HFSS project found.
	Opened HFSS-App.
	Opened HFSS desktop.
	Opened link to HFSS user project.
	Connected successfully.
	 :)	 :)	 :)	

Design "My Design - Qubit Example EPR" info:
	# eigenmodes    1
	# variations    1


# Calculate participation of the substrate for mode 1 

First, select which eigenmode to work on.<br>
Here the fundamental mode, mode 0, is the qubit

In [27]:
epr_hfss.set_mode(0) 

# Calculate the EPR p_dielectic
p_dielectic, (ℰ_substr, ℰ_total) = \
    epr_hfss.calc_p_electric_volume('substrate_si', 'AllObjects')

print(f'Energy in silicon substrate  = {100*p_dielectic:.1f}%')

Energy in silicon substrate  = 91.8%


Now, compute the electric energy stored in the vacuum 
Use the calculated total energy in all objects
so that we don't have to recompute it, since we 
computed it  above

In [26]:
p_vac, (ℰ_vac, ℰ_total) = epr_hfss.calc_p_electric_volume(
    'Region', E_total=ℰ_total) # Here, pass the precomputed E_total=ℰ_total

print(f'''Energy in vacuum = {100*p_vac:.1f}%

Since there are no other volumes,
the two energies should sum to one: {p_dielectic + p_vac}''')

Energy in vacuum = 8.2%

Since there are no other volumes,
the two energies should sum to one: 0.9999999999999954


In [7]:
? epr_hfss.calc_p_electric_volume

Signature:
 epr_hfss.calc_p_electric_volume(
    ['name_dielectric3D', "relative_to='AllObjects'", 'E_total=None'],
)
Docstring:
Calculate the dielectric energy-participatio ratio 
of a 3D object (one that has volume) relative to the dielectric energy of
a list of object objects.

This is as a function relative to another object or all objects.

When all objects are specified, this does not include any energy 
that might be stored in any lumped elements or lumped capacitors. 

Returns:
---------
    ℰ_object/ℰ_total, (ℰ_object, _total)
File:      c:\zkm-code\pyepr\pyepr\core.py
Type:      method


## Calculating the energies directly

Using lower level functions 

In [29]:
ℰ_total  = epr_hfss.calc_energy_electric(volume='AllObjects')
ℰ_substr = epr_hfss.calc_energy_electric(volume='Box1')
print(f'Energy in substrate = {100*ℰ_substr/ℰ_total:.1f}%')

Energy in substrate = 91.8%


In [30]:
?epr_hfss.calc_energy_electric

Signature:
epr_hfss.calc_energy_electric(
    ['variation=None', "volume='AllObjects'", 'smooth=False'],
)
Docstring:
Calculates two times the peak electric energy, or 4 times the RMS, :math:`4*\mathcal{E}_{\mathrm{elec}}`
(since we do not divide by 2 and use the peak phasors).

.. math::
    \mathcal{E}_{\mathrm{elec}}=\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}


volume : string | 'AllObjects'
smooth : bool | False 
    Smooth the electric field or not when performing calculation

Example use to calcualte the energy participation of a substrate

.. code-block python
    ℰ_total  = epr_hfss.calc_energy_electric(volume='AllObjects')
    ℰ_substr = epr_hfss.calc_energy_electric(volume='Box1')
    print(f'Energy in substrate = {100*ℰ_substr/ℰ_total:.1f}%')
File:      c:\zkm-code\pyepr\pyepr\core.py
Type:      method


# Using the Fields calculator in HFSS directly 

We will do the same calculation again, but now using the internals of `epr_hfss.calc_energy_electric` to demonstrate how the fields calcualtor object can be used for custom integrals and how the internals work. 

#### Using the HFSS Fields Calculator
The Fields calculator enables you to perform computations using basic field quantities. The calcu­lator will compute derived quantities from the general electric field solution; write field quantities to files, locate maximum and minimum field values, and perform other operations on the field solu­tion. 

The calculator does not perform the computations until a value is needed or is forced for a result. This makes it more efficient, saving computing resources and time; you can do all the calculations without regard to data storage of all the calculated points of the field. It is generally easier to do all the calculations first, then plot the results.

#### Direct calculation of 

\begin{align}
\mathcal{E}_{\mathrm{elec}}&=&\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}\;.
\end{align}



In [35]:
from pyEPR.core import *

self, volume = epr_hfss, 'AllObjects'

calcobject = CalcObject([], self.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_vol(name=volume)

E_total = A.evaluate(lv=self.get_lv()) 
# This command numerically evaluates and displays the
# results of calculator operations 
E_total

3.97856215141521

In [34]:
from pyEPR.core import *

self, volume = epr_hfss, 'substrate_si'

calcobject = CalcObject([], self.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_vol(name=volume)

E_subs = A.evaluate(lv=self.get_lv()) 
# This command numerically evaluates and displays the
# results of calculator operations 
E_subs

3.65565683193854

In [38]:
print(f'Energy in substrate: {100*E_subs/E_total:.1f}%')

Energy in substrate: 91.9%


# Summary  

We showed three levels of how to obtain the same result, using either the 
1. `pyEPR_HFSS.calc_p_electric_volume`
2. `pyEPR_HFSS.calc_energy_electric`
3. Or the fields calcualtor